In [1]:
import pandas as pd

data = pd.read_csv('data.csv')
data

,Timestep,S,I
0,1.1,0.9,0.1
1,2.2,0.8,NaN


In [5]:
results = pd.read_csv('results_petri/sample_results.csv', index_col=['timepoint_id', 'sample_id'])
results

,,beta_param,gamma_param,I_sol,R_sol,S_sol
timepoint_id,sample_id,,,,,
0,0,0.027670,0.128773,968.268921,31.608061,1.122878e+00
1,0,0.027670,0.128773,908.947876,92.051537,2.568984e-06
2,0,0.027670,0.128773,799.122375,201.876144,6.242269e-10
3,0,0.027670,0.128773,702.568359,298.431061,8.753653e-10
4,0,0.027670,0.128773,617.679749,383.319611,-6.266292e-10
0,1,0.026769,0.163082,960.723328,38.466320,1.810789e+00
1,1,0.026769,0.163082,887.172607,113.827003,7.685242e-06
2,1,0.026769,0.163082,753.672180,247.327606,1.312971e-11
3,1,0.026769,0.163082,640.259705,360.739166,5.487308e-10


In [23]:
def outputs_to_observations(timepoints, df: pd.DataFrame) -> dict[pd.DataFrame]:
    """Convert pyciemss outputs to data observations."""
    # Use groupby to create separate DataFrames
    grouped = df.groupby(level=1)

    # Create a dictionary of dataframes
    outputs = {level: group for level, group in grouped}
    observations = dict()
    for idx, observation in outputs.items():
        observation = observation.drop([k for k in observation.columns if '_sol' != k[-4:]],
                                      axis=1)
        observation = observation.rename(columns={k: k[:-4] for k in observation.columns})
        observation['Timestep'] = timepoints

        observations[idx] = observation
    return observations
        


In [24]:
outputs_to_observations([0.1, 0.2, 0.3, 0.4, 0.5], results)

{0:                                  I           R             S  Timestep
 timepoint_id sample_id                                                
 0            0          968.268921   31.608061  1.122878e+00       0.1
 1            0          908.947876   92.051537  2.568984e-06       0.2
 2            0          799.122375  201.876144  6.242269e-10       0.3
 3            0          702.568359  298.431061  8.753653e-10       0.4
 4            0          617.679749  383.319611 -6.266292e-10       0.5,
 1:                                  I           R             S  Timestep
 timepoint_id sample_id                                                
 0            1          960.723328   38.466320  1.810789e+00       0.1
 1            1          887.172607  113.827003  7.685242e-06       0.2
 2            1          753.672180  247.327606  1.312971e-11       0.3
 3            1          640.259705  360.739166  5.487308e-10       0.4
 4            1          543.914978  457.084808 -2.868339

In [25]:
from sympy import symbols, exp
from sympy.printing.mathml import mathml

I, S, kappa, beta_nc, beta_c, beta_s, k_2, t, t_1, k_1, t_0, N = symbols('I S kappa beta_nc beta_c beta_s k_2 t t_1 k_1 t_0 N')

expr = I*S*kappa*(beta_nc + (beta_c - beta_nc)/(1 + exp(-k_2*(-t + t_1))) + (-beta_c + beta_s)/(1 + exp(-k_1*(-t + t_0))))/N

mathml_str = mathml(expr)

print(mathml_str)


<apply><divide/><apply><times/><ci>I</ci><ci>S</ci><ci>&#954;</ci><apply><plus/><ci><mml:msub><mml:mi>&#946;</mml:mi><mml:mi>nc</mml:mi></mml:msub></ci><apply><divide/><apply><minus/><ci><mml:msub><mml:mi>&#946;</mml:mi><mml:mi>c</mml:mi></mml:msub></ci><ci><mml:msub><mml:mi>&#946;</mml:mi><mml:mi>nc</mml:mi></mml:msub></ci></apply><apply><plus/><cn>1</cn><apply><exp/><apply><minus/><apply><times/><ci><mml:msub><mml:mi>k</mml:mi><mml:mi>2</mml:mi></mml:msub></ci><apply><plus/><apply><minus/><ci>t</ci></apply><ci><mml:msub><mml:mi>t</mml:mi><mml:mi>1</mml:mi></mml:msub></ci></apply></apply></apply></apply></apply></apply><apply><divide/><apply><plus/><apply><minus/><ci><mml:msub><mml:mi>&#946;</mml:mi><mml:mi>c</mml:mi></mml:msub></ci></apply><ci><mml:msub><mml:mi>&#946;</mml:mi><mml:mi>s</mml:mi></mml:msub></ci></apply><apply><plus/><cn>1</cn><apply><exp/><apply><minus/><apply><times/><ci><mml:msub><mml:mi>k</mml:mi><mml:mi>1</mml:mi></mml:msub></ci><apply><plus/><apply><minus/><ci>t</